In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')
products = pd.read_csv('Products.csv')

# Merge datasets for unified analysis
data = pd.merge(transactions, customers, on='CustomerID', how='left')
data = pd.merge(data, products, on='ProductID', how='left')

# Initial exploration
print("Dataset Info:")
print(data.info())
print("\nSummary Statistics:")
print(data.describe())
print("\nMissing Values:")
print(data.isnull().sum())

# Convert TransactionDate and SignupDate to datetime
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['SignupDate'] = pd.to_datetime(data['SignupDate'])

# Feature engineering: Extract year and month
data['TransactionYear'] = data['TransactionDate'].dt.year
data['TransactionMonth'] = data['TransactionDate'].dt.month
data['SignupYear'] = data['SignupDate'].dt.year

# Exploratory Analysis
# 1. Distribution of transactions by region
plt.figure(figsize=(10, 6))
sns.countplot(data=data, x='Region', order=data['Region'].value_counts().index)
plt.title('Transaction Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Count')
plt.show()

# 2. Revenue contribution by product category
category_revenue = data.groupby('Category')['TotalValue'].sum().sort_values(ascending=False)
category_revenue.plot(kind='bar', figsize=(10, 6), title='Revenue Contribution by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Total Revenue')
plt.show()

# 3. Trend of transactions over time
plt.figure(figsize=(12, 6))
monthly_transactions = data.groupby(['TransactionYear', 'TransactionMonth'])['TransactionID'].count()
monthly_transactions.plot(title='Monthly Transaction Trends')
plt.xlabel('Year-Month')
plt.ylabel('Number of Transactions')
plt.show()

# 4. Average transaction value by region
region_avg_transaction = data.groupby('Region')['TotalValue'].mean().sort_values(ascending=False)
region_avg_transaction.plot(kind='bar', figsize=(10, 6), title='Average Transaction Value by Region')
plt.xlabel('Region')
plt.ylabel('Average Transaction Value (USD)')
plt.show()

# 5. Top customers by revenue contribution
top_customers = data.groupby('CustomerName')['TotalValue'].sum().sort_values(ascending=False).head(10)
top_customers.plot(kind='bar', figsize=(10, 6), title='Top 10 Customers by Revenue Contribution')
plt.xlabel('Customer Name')
plt.ylabel('Total Revenue (USD)')
plt.show()

# Derive Business Insights
print("\n### Business Insights ###")
print("1. The majority of transactions are concentrated in certain regions, with North America leading.")
print("2. Product categories significantly impact revenue, with certain categories contributing the majority of the revenue.")
print("3. There is a noticeable seasonal trend in transactions, with peaks around certain months.")
print("4. Customers from Asia have the highest average transaction value, indicating premium spending behavior.")
print("5. The top 10 customers contribute a significant portion of revenue, highlighting the importance of key accounts.")

In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Load datasets
customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')
products = pd.read_csv('Products.csv')

# Merge datasets
data = pd.merge(transactions, customers, on='CustomerID', how='left')
data = pd.merge(data, products, on='ProductID', how='left')

# Aggregate transaction data to create customer profiles
customer_profiles = data.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],
    'Quantity': 'sum',
    'Category': lambda x: x.value_counts().index[0],  # Most frequent category
}).reset_index()

# Rename columns for clarity
customer_profiles.columns = ['CustomerID', 'TotalSpend', 'AvgSpend', 'TotalQuantity', 'MostFrequentCategory']

# Encode categorical data (MostFrequentCategory)
customer_profiles = pd.get_dummies(customer_profiles, columns=['MostFrequentCategory'])

# Normalize numerical features for similarity calculation
scaler = MinMaxScaler()
numerical_features = ['TotalSpend', 'AvgSpend', 'TotalQuantity']
customer_profiles[numerical_features] = scaler.fit_transform(customer_profiles[numerical_features])

# Calculate cosine similarity
customer_features = customer_profiles.drop('CustomerID', axis=1)
similarity_matrix = cosine_similarity(customer_features)

# Recommendation function
def get_similar_customers(customer_id, top_n=3):
    # Get the index of the customer in the matrix
    customer_idx = customer_profiles[customer_profiles['CustomerID'] == customer_id].index[0]
    
    # Get similarity scores for the customer
    similarity_scores = similarity_matrix[customer_idx]
    
    # Sort by similarity score
    similar_customers_idx = similarity_scores.argsort()[::-1][1:top_n+1]
    
    # Retrieve similar customers and their scores
    similar_customers = customer_profiles.iloc[similar_customers_idx][['CustomerID']].copy()
    similar_customers['SimilarityScore'] = similarity_scores[similar_customers_idx]
    
    return similar_customers

# Generate recommendations for the first 20 customers
lookalike_results = {}
for customer_id in customer_profiles['CustomerID'].head(20):
    similar_customers = get_similar_customers(customer_id)
    lookalike_results[customer_id] = similar_customers.values.tolist()

# Save results to CSV
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_results.keys(),
    'SimilarCustomers': lookalike_results.values()
})
lookalike_df.to_csv('Lookalike.csv', index=False)

# Display sample recommendations
print(lookalike_df.head())

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load datasets
customers = pd.read_csv('Customers.csv')
transactions = pd.read_csv('Transactions.csv')

# Merge datasets
data = pd.merge(transactions, customers, on='CustomerID', how='left')

# Aggregate transaction data to create customer profiles
customer_profiles = data.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],  # Total and Average Spend
    'Quantity': 'sum',             # Total Quantity Purchased
}).reset_index()

# Rename columns for clarity
customer_profiles.columns = ['CustomerID', 'TotalSpend', 'AvgSpend', 'TotalQuantity']

# Merge with customer profile information (Region, SignupDate)
customer_profiles = pd.merge(customer_profiles, customers[['CustomerID', 'Region']], on='CustomerID', how='left')

# Encode categorical data (Region)
encoder = OneHotEncoder()
encoded_region = encoder.fit_transform(customer_profiles[['Region']]).toarray()
encoded_region_df = pd.DataFrame(encoded_region, columns=encoder.get_feature_names_out(['Region']))
customer_profiles = pd.concat([customer_profiles, encoded_region_df], axis=1)

# Drop unnecessary columns
customer_profiles = customer_profiles.drop(['CustomerID', 'Region'], axis=1)

# Normalize numerical features
scaler = StandardScaler()
customer_profiles_scaled = scaler.fit_transform(customer_profiles)

# Clustering with K-Means
db_scores = []
cluster_range = range(2, 11)
for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(customer_profiles_scaled)
    db_index = davies_bouldin_score(customer_profiles_scaled, clusters)
    db_scores.append(db_index)

# Find optimal number of clusters (minimum DB Index)
optimal_k = cluster_range[np.argmin(db_scores)]
print(f"Optimal Number of Clusters: {optimal_k}")

# Perform clustering with optimal number of clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
customer_profiles['Cluster'] = kmeans.fit_predict(customer_profiles_scaled)

# Visualize DB Index
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, db_scores, marker='o', linestyle='-', color='b')
plt.title('Davies-Bouldin Index for Different Cluster Numbers')
plt.xlabel('Number of Clusters')
plt.ylabel('DB Index')
plt.show()

# Visualize clusters using PCA
pca = PCA(n_components=2)
customer_profiles_pca = pca.fit_transform(customer_profiles_scaled)
plt.figure(figsize=(12, 8))
sns.scatterplot(x=customer_profiles_pca[:, 0], y=customer_profiles_pca[:, 1], hue=customer_profiles['Cluster'], palette='Set2', s=100)
plt.title('Customer Segmentation Clusters (PCA Projection)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster')
plt.show()

# Evaluate and report metrics
db_index_optimal = davies_bouldin_score(customer_profiles_scaled, customer_profiles['Cluster'])
print(f"Davies-Bouldin Index for {optimal_k} Clusters: {db_index_optimal:.2f}")
